# Análisis de calidad del aire
1. Carga los datos demográficos en una tabla utilizando pandas, utilizando el mismo CSV de demografía del ejemplo estudiado en clases.

Importa directamente usando esta url con el siguiente código:

url = "https://public.opendatasoft.com/explore/dataset/us-cities-demographics/download/?format=csv&timezone=Europe/Berlin&lang=en&use_labels_for_header=true&csv_separator=%3B"
data = pd.read_csv(url, sep=';')

2. Parsea los datos de calidad del aire para cada ciudad en la tabla demográfica obteniendo la información con la API https://api-ninjas.com/api/airquality. Crea una tabla de dimensiones utilizando pandas para almacenar estos datos.

Toma el elemento concentration de cada entrada por fila.

3. Limpia los datos demográficos realizando las siguientes acciones:

Elimina las columnas: Race, Count y Number of Veterans.
Elimina las filas duplicadas.

4. Crea una base de datos en SQLite, carga las dos tablas procesadas ahí

5. Aplica joins, y agregaciones para verificar si las ciudades más pobladas tienen la peor calidad del aire. (muestra las primeras 10 colúmnas y con eso responde la pregunta)

Crea un script para el ejercicio 3, luego en un archivo markdown escribe la query SQL que utilizaste para responder la pregunta y escribe una explicación detallada de tu interpretación de los resultados, este dos archivo súbelo aquí. (Adjunta también la base de datos sqlite).

Para los ejercicios 1 y 2 usa la siguiente plantilla:

import pandas as pd
from typing import Set


def ej_1_cargar_datos_demograficos() -> pd.DataFrame:
    pass

def ej_2_cargar_calidad_aire(ciudades: Set[str]) -> None:
    pass
Y usa los siguientes tests:

### Estos tests pueden fallar, sólo están aquí como referencia ya que debido a la actualización de valores por parte de la api, no se pueden mantener fijos para evaluar en todo momento al mismo valor.
from hashlib import sha256
from soluciones import (
    ej_1_cargar_datos_demograficos,
    ej_2_cargar_calidad_aire,
)
import pandas as pd


def _hash(data):
    return sha256(str(data).encode("utf-8")).hexdigest()


def test_sol_1():
    df = ej_1_cargar_datos_demograficos()
    idxs = [1995, 1360, 982, 2264, 2096, 1733, 1804, 2025, 2070, 507]
    
    selected_rows = df.loc[idxs].values
    assert _hash(selected_rows) == "567b67390efd8da8091f6f86da9f5e76b30d1b7dcb25bd7d9b87bcb757b2c571"
    
    
def test_sol_2():
    df = ej_1_cargar_datos_demograficos()
    ej_2_cargar_calidad_aire(set(df["City"].tolist()))
    
    ciudades_df = pd.read_csv("ciudades.csv")
    
    actual = ciudades_df.loc[:9].to_dict()
    
    expected = {
        'CO': {0: 250.34, 1: 287.06, 2: 247.0, 3: 280.38, 4: 323.77, 5: 243.66, 6: 173.57, 7: 211.95, 8: 263.69, 9: 260.35},
        'NO2': {0: 3.43, 1: 0.76, 2: 0.56, 3: 1.06, 4: 1.67, 5: 0.91, 6: 0.23, 7: 0.84, 8: 0.8, 9: 1.71},
        'O3': {0: 167.37, 1: 103.0, 2: 41.13, 3: 98.71, 4: 86.55, 5: 100.14, 6: 94.41, 7: 105.86, 8: 100.14, 9: 130.18},
        'SO2': {0: 2.92, 1: 2.3, 2: 0.19, 3: 1.1, 4: 6.32, 5: 1.27, 6: 0.38, 7: 0.24, 8: 0.4, 9: 5.36},
        'PM2.5': {0: 17.78, 1: 6.06, 2: 1.79, 3: 4.08, 4: 2.64, 5: 5.43, 6: 12.62, 7: 1.67, 8: 4.49, 9: 6.21},
        'PM10': {0: 26.26, 1: 6.37, 2: 1.85, 3: 4.47, 4: 2.95, 5: 5.68, 6: 48.06, 7: 1.79, 8: 4.66, 9: 6.76},
        'overall_aqi': {0: 220, 1: 170, 2: 34, 3: 159, 4: 128, 5: 162, 6: 148, 7: 177, 8: 162, 9: 205},
        'city': {0: 'Perris', 1: 'Mount Vernon', 2: 'Mobile', 3: 'Dale City', 4: 'Maple Grove', 5: 'Muncie', 6: 'San Clemente', 7: 'Providence', 8: 'Norman', 9: 'Hoover'}
    }
    
    assert expected == actual
Debido a que este laboratorio tiene una solución más abierta que los demás, no existen tests para todos los pasos, es por eso que debes detallar tu solución en el markdown para faciliar la revisión a mano.

In [63]:
import pandas as pd
import requests
import sqlite3

In [64]:
# Carga de datos 
url = "https://public.opendatasoft.com/explore/dataset/us-cities-demographics/download/?format=csv&timezone=Europe/Berlin&lang=en&use_labels_for_header=true&csv_separator=%3B"
data_demo = pd.read_csv(url, sep=';')

In [65]:
data_demo

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Denver,Colorado,34.1,341137.0,341408.0,682545,29363.0,113222.0,2.33,CO,Black or African-American,72288
1,Provo,Utah,23.6,56231.0,59027.0,115258,2177.0,10925.0,3.28,UT,American Indian and Alaska Native,1916
2,Hampton,Virginia,35.5,66214.0,70240.0,136454,19638.0,6204.0,2.48,VA,Hispanic or Latino,7513
3,Birmingham,Alabama,35.6,102122.0,112789.0,214911,13212.0,8258.0,2.21,AL,Asian,1500
4,Greeley,Colorado,31.0,50792.0,50091.0,100883,4294.0,11480.0,2.75,CO,White,92874
...,...,...,...,...,...,...,...,...,...,...,...,...
2886,Mesa,Arizona,36.9,234998.0,236835.0,471833,31808.0,57492.0,2.68,AZ,White,413010
2887,Corpus Christi,Texas,35.0,160488.0,163594.0,324082,25078.0,30834.0,2.69,TX,Hispanic or Latino,200737
2888,Bismarck,North Dakota,38.0,34675.0,35565.0,70240,4145.0,2064.0,2.11,ND,Hispanic or Latino,1667
2889,Orem,Utah,26.1,48695.0,45762.0,94457,2828.0,12808.0,3.26,UT,American Indian and Alaska Native,976


In [68]:
data_demo['City'].head(100)

0          Denver
1           Provo
2         Hampton
3      Birmingham
4         Greeley
         ...     
95     Huntsville
96        Turlock
97    Gainesville
98      Champaign
99      Inglewood
Name: City, Length: 100, dtype: object

In [69]:
# Parsear datos de calidad del aire
air_quality_data = []

for city in data_demo['City'].head(100):
    api_url = f"https://api.api-ninjas.com/v1/airquality?city={city}"
    response = requests.get(api_url, headers={'X-Api-Key': 'UQzvNTUxwc8yrG0321yCMw==BiVhGOKxnwS1tk18'})
    
    if response.status_code == 200:
        air_quality_data.append(response.json())
    else:
        air_quality_data.append({'city': city, 'concentration': None})




In [70]:
## Tome 100 datos solamente, porque los datos de cada ciudad son demasiado, por lo que se demora mucho en obtenerlos
# Y tambien cuestan mucho consumo de api x ciudad. Por lo que solo se toman los siguientes datos = 
air_quality_data

[{'CO': {'concentration': 293.73, 'aqi': 3},
  'NO2': {'concentration': 16.79, 'aqi': 20},
  'O3': {'concentration': 67.95, 'aqi': 77},
  'SO2': {'concentration': 5.07, 'aqi': 7},
  'PM2.5': {'concentration': 4.29, 'aqi': 13},
  'PM10': {'concentration': 6.14, 'aqi': 5},
  'overall_aqi': 77},
 {'CO': {'concentration': 263.69, 'aqi': 2},
  'NO2': {'concentration': 8.23, 'aqi': 10},
  'O3': {'concentration': 64.37, 'aqi': 66},
  'SO2': {'concentration': 1.94, 'aqi': 2},
  'PM2.5': {'concentration': 5.88, 'aqi': 19},
  'PM10': {'concentration': 8.52, 'aqi': 7},
  'overall_aqi': 66},
 {'CO': {'concentration': 290.39, 'aqi': 3},
  'NO2': {'concentration': 13.2, 'aqi': 16},
  'O3': {'concentration': 67.23, 'aqi': 75},
  'SO2': {'concentration': 1.27, 'aqi': 1},
  'PM2.5': {'concentration': 4.05, 'aqi': 13},
  'PM10': {'concentration': 10.32, 'aqi': 9},
  'overall_aqi': 75},
 {'CO': {'concentration': 270.37, 'aqi': 3},
  'NO2': {'concentration': 43.53, 'aqi': 54},
  'O3': {'concentration': 16

In [83]:
# Convertir los datos a un DataFrame
calidad_aire = pd.json_normalize(air_quality_data)
calidad_aire


,overall_aqi,CO.concentration,CO.aqi,NO2.concentration,NO2.aqi,O3.concentration,O3.aqi,SO2.concentration,SO2.aqi,PM2.5.concentration,PM2.5.aqi,PM10.concentration,PM10.aqi
0,77,293.73,3,16.79,20,67.95,77,5.07,7,4.29,13,6.14,5
1,66,263.69,2,8.23,10,64.37,66,1.94,2,5.88,19,8.52,7
2,75,290.39,3,13.20,16,67.23,75,1.27,1,4.05,13,10.32,9
3,54,270.37,3,43.53,54,16.27,13,6.91,10,3.11,10,3.81,3
4,38,317.10,3,11.65,14,45.42,38,1.09,1,10.07,32,10.96,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,84,236.99,2,3.56,4,70.10,84,1.39,2,4.76,15,7.16,6
96,19,307.08,3,11.31,14,22.53,19,0.92,1,4.58,14,5.94,5
97,107,223.64,2,1.25,1,77.96,107,0.15,0,0.99,3,1.61,1
98,57,257.02,2,2.31,2,61.51,57,0.55,0,2.99,9,3.62,3


In [85]:
# Crear un nuevo DataFrame solo con las columnas 'concentration'
calidad_aire_concentration = calidad_aire.filter(like='concentration')

# Imprimir el DataFrame
calidad_aire_concentration


,CO.concentration,NO2.concentration,O3.concentration,SO2.concentration,PM2.5.concentration,PM10.concentration
0,293.73,16.79,67.95,5.07,4.29,6.14
1,263.69,8.23,64.37,1.94,5.88,8.52
2,290.39,13.20,67.23,1.27,4.05,10.32
3,270.37,43.53,16.27,6.91,3.11,3.81
4,317.10,11.65,45.42,1.09,10.07,10.96
...,...,...,...,...,...,...
95,236.99,3.56,70.10,1.39,4.76,7.16
96,307.08,11.31,22.53,0.92,4.58,5.94
97,223.64,1.25,77.96,0.15,0.99,1.61
98,257.02,2.31,61.51,0.55,2.99,3.62


In [105]:
# Eliminar las filas duplicadas
data_demo_ac = data_demo.drop_duplicates() #data actualizada

In [107]:
# Eliminar las columnas 'Race', 'Count' y 'Number of Veterans'
data_ac = data_demo.drop(columns=['Race', 'Count', 'Number of Veterans'])

In [108]:
data_ac

,City,State,Median Age,Male Population,Female Population,Total Population,Foreign-born,Average Household Size,State Code
0,Denver,Colorado,34.1,341137.0,341408.0,682545,113222.0,2.33,CO
1,Provo,Utah,23.6,56231.0,59027.0,115258,10925.0,3.28,UT
2,Hampton,Virginia,35.5,66214.0,70240.0,136454,6204.0,2.48,VA
3,Birmingham,Alabama,35.6,102122.0,112789.0,214911,8258.0,2.21,AL
4,Greeley,Colorado,31.0,50792.0,50091.0,100883,11480.0,2.75,CO
...,...,...,...,...,...,...,...,...,...
2886,Mesa,Arizona,36.9,234998.0,236835.0,471833,57492.0,2.68,AZ
2887,Corpus Christi,Texas,35.0,160488.0,163594.0,324082,30834.0,2.69,TX
2888,Bismarck,North Dakota,38.0,34675.0,35565.0,70240,2064.0,2.11,ND
2889,Orem,Utah,26.1,48695.0,45762.0,94457,12808.0,3.26,UT


In [110]:
# Recordando que decidimos trabajar solo con los primeros 100 datos debido a las apis anteriores, tenemos que = 
data_ac.head(100)

,City,State,Median Age,Male Population,Female Population,Total Population,Foreign-born,Average Household Size,State Code
0,Denver,Colorado,34.1,341137.0,341408.0,682545,113222.0,2.33,CO
1,Provo,Utah,23.6,56231.0,59027.0,115258,10925.0,3.28,UT
2,Hampton,Virginia,35.5,66214.0,70240.0,136454,6204.0,2.48,VA
3,Birmingham,Alabama,35.6,102122.0,112789.0,214911,8258.0,2.21,AL
4,Greeley,Colorado,31.0,50792.0,50091.0,100883,11480.0,2.75,CO
...,...,...,...,...,...,...,...,...,...
95,Huntsville,Alabama,38.1,91764.0,97350.0,189114,12691.0,2.18,AL
96,Turlock,California,36.2,33190.0,39103.0,72293,14686.0,2.76,CA
97,Gainesville,Florida,26.0,60803.0,69330.0,130133,15272.0,2.33,FL
98,Champaign,Illinois,28.7,43326.0,42760.0,86086,12261.0,2.25,IL


In [112]:
calidad_aire_concentration

,CO.concentration,NO2.concentration,O3.concentration,SO2.concentration,PM2.5.concentration,PM10.concentration
0,293.73,16.79,67.95,5.07,4.29,6.14
1,263.69,8.23,64.37,1.94,5.88,8.52
2,290.39,13.20,67.23,1.27,4.05,10.32
3,270.37,43.53,16.27,6.91,3.11,3.81
4,317.10,11.65,45.42,1.09,10.07,10.96
...,...,...,...,...,...,...
95,236.99,3.56,70.10,1.39,4.76,7.16
96,307.08,11.31,22.53,0.92,4.58,5.94
97,223.64,1.25,77.96,0.15,0.99,1.61
98,257.02,2.31,61.51,0.55,2.99,3.62


In [114]:
#Relacionando las dos bases de datos entre si, tenemos que = 
datos_completos_relacionados = pd.concat([data_ac.head(100), calidad_aire_concentration], axis=1)
datos_completos_relacionados

,City,State,Median Age,Male Population,Female Population,Total Population,Foreign-born,Average Household Size,State Code,CO.concentration,NO2.concentration,O3.concentration,SO2.concentration,PM2.5.concentration,PM10.concentration
0,Denver,Colorado,34.1,341137.0,341408.0,682545,113222.0,2.33,CO,293.73,16.79,67.95,5.07,4.29,6.14
1,Provo,Utah,23.6,56231.0,59027.0,115258,10925.0,3.28,UT,263.69,8.23,64.37,1.94,5.88,8.52
2,Hampton,Virginia,35.5,66214.0,70240.0,136454,6204.0,2.48,VA,290.39,13.20,67.23,1.27,4.05,10.32
3,Birmingham,Alabama,35.6,102122.0,112789.0,214911,8258.0,2.21,AL,270.37,43.53,16.27,6.91,3.11,3.81
4,Greeley,Colorado,31.0,50792.0,50091.0,100883,11480.0,2.75,CO,317.10,11.65,45.42,1.09,10.07,10.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Huntsville,Alabama,38.1,91764.0,97350.0,189114,12691.0,2.18,AL,236.99,3.56,70.10,1.39,4.76,7.16
96,Turlock,California,36.2,33190.0,39103.0,72293,14686.0,2.76,CA,307.08,11.31,22.53,0.92,4.58,5.94
97,Gainesville,Florida,26.0,60803.0,69330.0,130133,15272.0,2.33,FL,223.64,1.25,77.96,0.15,0.99,1.61
98,Champaign,Illinois,28.7,43326.0,42760.0,86086,12261.0,2.25,IL,257.02,2.31,61.51,0.55,2.99,3.62


# Para cumplir con la peticion del ejercicio.
```python
# Importar las bibliotecas necesarias
import sqlite3
import pandas as pd

# Crear una conexión a la base de datos SQLite
conn = sqlite3.connect('mi_base_de_datos.db')

# Cargar las dos tablas procesadas en la base de datos
df.to_sql('tabla1', conn, if_exists='replace', index=False)
data_demo.to_sql('tabla2', conn, if_exists='replace', index=False)

# Crear una consulta SQL para aplicar joins y agregaciones
sql = """
SELECT *
FROM tabla1
JOIN tabla2 ON tabla1.ciudad = tabla2.ciudad
GROUP BY tabla1.ciudad
ORDER BY tabla2.poblacion DESC
LIMIT 10
"""

# Ejecutar la consulta SQL y guardar el resultado en un DataFrame
resultado = pd.read_sql_query(sql, conn)

# Mostrar las primeras 10 filas del resultado
print(resultado.head(10))
```
#### Los pytest se hicieron en local

